# Amazon Nova for Object Detection and Bounding Box Annotation

Amazon Nova models have powerful capabilities for image grounding and object detection tasks across a wide range of domains. This notebook demonstrates how to use Amazon Nova models to detect objects in images and create bounding box annotations with high precision.

## What you'll learn in this notebook

- How to prompt Nova models for object detection with bounding boxes
- How to parse and visualize bounding box coordinates
- How to customize detection for specific object categories
- How to handle different image types and scenarios

## Real-world use cases for object detection

Object detection with Nova models can be applied to numerous industry applications:

1. **Retail and E-commerce**
   - Product identification in shelf images
   - Inventory management and out-of-stock detection
   - Visual search and product recommendations

2. **Manufacturing and Quality Control**
   - Defect detection in production lines
   - Component identification and verification
   - Assembly verification and quality assurance

3. **Security and Surveillance**
   - Person and vehicle detection
   - Suspicious object identification
   - Intrusion detection systems

4. **Healthcare and Medical Imaging**
   - Anomaly detection in medical scans
   - Cell and tissue identification
   - Medical device positioning verification

5. **Agriculture**
   - Crop disease detection
   - Ripeness assessment
   - Weed identification

6. **Smart Cities**
   - Traffic monitoring and analysis
   - Parking space management
   - Infrastructure inspection

## Technical details

The output bounding box coordinates are in scale [0, 1000), which you need to rescale to the original image size. This notebook demonstrates the full pipeline from image input to visualized detection output."

In [ ]:
%pip install boto3 Pillow

In [ ]:
import base64
import boto3
import io
import json
import re

from PIL import Image, ImageDraw, ImageFont



modelId = "us.amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"
bedrock_rt = boto3.client("bedrock-runtime", region_name="us-west-2") 

Define a helper function for better loading model output as json.

In [ ]:

def safe_json_load(json_string):
    try:
        json_string = re.sub(r"\s", "", json_string)
        json_string = re.sub(r"\(", "[", json_string)
        json_string = re.sub(r"\)", "]", json_string)
        bbox_set = {}
        for b in re.finditer(r"\[\d+,\d+,\d+,\d+\]", json_string):
            if b.group(0) in bbox_set:
                json_string = json_string[:bbox_set[b.group(0)][1]] + "}]"
                break
            bbox_set[b.group(0)] = (b.start(), b.end())
        else:
            json_string = json_string[:bbox_set[b.group(0)][1]] + "}]"
        json_string = re.sub(r"\]\},\]$", "]}]", json_string)
        json_string = re.sub(r"\]\],\[\"", "]},{\"", json_string)
        json_string = re.sub(r"\]\],\[\{\"", "]},{\"", json_string)
        return json.loads(json_string)
    except Exception as e:
        print(e)
        return []

Invoke Amazon Nova for image detection, you need to define the detection task by prompt. Here is the prompt for specific object detection, and you can define you own prompt based on your needs.

In [ ]:
def detection(image_path, category_list, image_short_size=360):
    image_pil = Image.open(image_path)
    width, height = image_pil.size

    ratio = image_short_size/ min(width, height)
    width = round(ratio * width)
    height = round(ratio * height)

    image_pil = image_pil.resize((width, height), resample=Image.Resampling.LANCZOS)
    buffer = io.BytesIO()
    image_pil.save(buffer, format="webp", quality=90)
    image_data = buffer.getvalue()


    category_str = ",".join([f'"{category}"' for category in category_list])
    
    prompts = """
Detect bounding box of objects in the image, only detect %s category objects with high confidence, output in a list of bounding box format.
Output example:
[
    {"%s": [x1, y1, x2, y2]},
    ...
]
""" % (category_str, category_list[0])
    prefill="""[
    {"
"""

    print("Input Prompt:\n", prompts)

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": 'webp',
                        "source": {
                            "bytes": image_data,
                        }
                    }
                },
                {
                    "text": prompts
                },
            ],
        },
        {
            "role": "assistant",
            "content": [
                {
                    "text": prefill
                },
            ],
        }
    ]

    response = bedrock_rt.converse(
        modelId=modelId, messages=messages,
        inferenceConfig={
            "temperature": 0.0,
            "maxTokens": 1024,
        },
    )
    output = prefill + response.get('output')["message"]["content"][0]["text"]

    result = safe_json_load(output)

    color_list = [
        'blue',
        'green',
        'yellow',
        'red',
        'orange',
        'pink',
        'purple',
    ]

    print("Result:\n")
    for idx, item in enumerate(result):
        label = next(iter(item))
        bbox = item[label]
        x1, y1, x2, y2 = bbox
        if x1 >= x2 or y1 >= y2:
            continue
        w, h = image_pil.size
        x1 = x1 / 1000 * w
        x2 = x2 / 1000 * w
        y1 = y1 / 1000 * h
        y2 = y2 / 1000 * h
        bbox = (x1, y1, x2, y2)
        bbox = list(map(round, bbox))
        print(f"Detect <{label}> in {bbox}")
        # draw bounding box
        draw = ImageDraw.Draw(image_pil)
        color = color_list[idx % len(color_list)]
        draw.rectangle(bbox, outline=color, width=2)
        draw.text((x1 + 4, y1 + 2), label, fill=color)
    return image_pil


Here are some examples:

In [ ]:
detection(
    "./image/bottle.webp", 
    category_list=["bottle"], 
)

In [ ]:
detection(
    "./image/cat_dog_car.webp", 
    category_list=["car", "dog", "cat"], 
)

In [ ]:
detection(
    "./image/cakes.webp", 
    category_list=["cake"], 
)

In [ ]:
detection(
    "./image/unripe_strawberry.webp", 
    category_list=["unripe_strawberry"], 
)

# Conclusion

In this notebook, we explored how to use Amazon Nova models for object detection and bounding box annotation. Here's what we learned:

1. Nova models can detect objects in images with high accuracy
2. We can customize detection by specifying which categories to look for
3. The models return coordinates that we need to scale to the original image size
4. We can visualize detections by drawing bounding boxes and labels
5. This approach works for different types of images and object categories